Replicate [Dynamic Return Dependencies Across Industries: A Machine Learning Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3120110&download=yes) by David Rapach, Jack Strauss, Jun Tu and Guofu Zhou.

1) Use Keras NNs instead of linear regression

2) Add additional variables, 3 and 12-month MA, interest rate change, yield curve, Mkt-RF, seasonal dummy variables. With cross-validation and regularization we hope to do that without overfitting and possibly produce a better result.


In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import pandas_datareader.data as datareader
import time 
import datetime
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf
tf.set_random_seed(1764)
print(tf.__version__)
# confirm GPU is in use
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))
    
import keras
from keras.layers.core import Dense, Activation
from keras.layers import Input
from keras.models import Model

from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l1
from keras.models import Sequential
from keras.models import load_model

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


1.6.0
[[22. 28.]
 [49. 64.]]


Using TensorFlow backend.


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

    
# add rates data from FRED
start_date = datetime.datetime(1926, 9, 1)
end_date = datetime.datetime(2017, 12, 1)
TB3MS = datareader.DataReader("TB3MS", "fred", start_date, end_date)
TB3MS['yyyymm'] = TB3MS.index.strftime('%Y%m')
TB3MS['yyyymm'] = [int(datestr) for datestr in TB3MS['yyyymm']]
TB3MS=TB3MS.set_index(['yyyymm'])
data['3month']=TB3MS['TB3MS']

GS10 =  datareader.DataReader("GS10", "fred", start_date, end_date)
GS10['yyyymm'] = GS10.index.strftime('%Y%m')
GS10['yyyymm'] = [int(datestr) for datestr in GS10['yyyymm']]
GS10=GS10.set_index(['yyyymm'])
data['10year']=GS10['GS10']

data['curve'] = data['10year'] - data['3month']
data['10year'] = data['10year'].diff() # first difference 10-year yield
data['3month'] = data['3month'].diff() # first difference 3-month

data['month'] = data.index  % 100 # for possible seasonality
month_dummy = keras.utils.to_categorical(data['month'])
for i in range(month_dummy.shape[1]):
    data['month_dummy_%02d' % i] = month_dummy[:,i]
data = data.drop(columns=['month'])    
#data[['month_dummy_%02d' % i for i in range(12)]]

data['Mkt-RF'] = rfdata['Mkt-RF']

for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
    data[ind+".12m"] = pd.rolling_mean(data[ind],12)

for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)
print(list(data.columns))
data[['3month', '10year', 'curve', 'Mkt-RF',]]


Loading data...
(697, 145)
['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other', '3month', '10year', 'curve', 'month_dummy_00', 'month_dummy_01', 'month_dummy_02', 'month_dummy_03', 'month_dummy_04', 'month_dummy_05', 'month_dummy_06', 'month_dummy_07', 'month_dummy_08', 'month_dummy_09', 'month_dummy_10', 'month_dummy_11', 'month_dummy_12', 'Mkt-RF', 'Food.3m', 'Beer.3m', 'Smoke.3m', 'Games.3m', 'Books.3m', 'Hshld.3m', 'Clths.3m', 'Hlth.3m', 'Chems.3m', 'Txtls.3m', 'Cnstr.3m', 'Steel.3m', 'FabPr.3m', 'ElcEq.3m', 'Autos.3m', 'Carry.3m', 'Mines.3m', 'Coal.3m', 'Oil.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'BusEq.3m', 'Paper.3m', 'Trans.3m', 'Whlsl.3m', 'Rtail.3m', 'Meals.3m', 'Fin.3m', 'Other.3m', '3month.3m', '10year.3m', 'curve.3m', 'Mkt-RF.3m', 'Food.12m', 'Beer.12m', 'Smoke.12m',

,3month,10year,curve,Mkt-RF
yyyymm,,,,
195912,0.34,0.16,0.20,2.45
196001,-0.14,0.03,0.37,-6.98
196002,-0.39,-0.23,0.53,1.17
196003,-0.65,-0.24,0.94,-1.63
196004,-0.08,0.03,1.05,-1.71
196005,0.06,0.07,1.06,3.12
196006,-0.83,-0.20,1.69,2.08
196007,-0.16,-0.25,1.60,-2.37
196008,0.00,-0.10,1.50,3.01


In [3]:
#data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,...,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000
mean,0.688666,0.727030,0.985079,0.732095,0.532253,0.564333,0.690387,0.665825,0.552367,0.687145,...,0.515968,0.729928,0.622970,0.534806,0.601090,0.631076,0.698235,0.728766,0.637547,0.396628
std,4.308660,5.058992,6.032324,7.128170,5.780362,4.728000,6.355251,4.897557,5.482363,6.970961,...,4.607931,6.486956,6.698787,5.021876,5.707154,5.571040,5.334178,6.065564,5.381389,5.771655
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.630000,-2.080000,-2.740000,-3.390000,-2.600000,-2.030000,-2.800000,-2.230000,-2.750000,-3.170000,...,-2.110000,-3.050000,-3.220000,-2.400000,-2.780000,-2.560000,-2.380000,-2.840000,-2.400000,-2.930000
50%,0.740000,0.750000,1.270000,0.940000,0.510000,0.750000,0.700000,0.760000,0.720000,0.640000,...,0.590000,1.010000,0.670000,0.710000,0.900000,0.940000,0.540000,1.080000,0.870000,0.540000
75%,3.070000,3.690000,4.660000,5.260000,3.640000,3.540000,4.310000,3.550000,3.760000,4.480000,...,3.360000,4.260000,4.630000,3.460000,4.040000,3.880000,3.980000,4.300000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# Run LASSO, then OLS on selected variables

# We get better R-squareds across the board in-sample, unsurprisingly

X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(696, 115)

In [6]:
def subset_selection(X, Y, model_aic, verbose=False):
    
    global responses
    global response_reverse_dict
    global predictors
    global predictor_reverse_dict
    
    coef_dict = {}
    for response_index, response in enumerate(responses):
        y = Y[:,response_reverse_dict[response]]
        
        model_aic.fit(X, y)

        coef_dict[response] = [predstr for i, predstr in enumerate(predictors) if model_aic.coef_[i] !=0]
        #y_response = model_aic.responseict(X)
        # print ("In-sample LASSO R-squared: %.6f" % r2_score(y, y_response))
        if verbose:
            print("LASSO variables selected for %s: " % response)
            print(coef_dict[response])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response + ", using all")
                print("---")
            coef_dict[response] = predictors            
        # fit OLS vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        if verbose:
            print("Running OLS for " + response + " against " + str(coef_dict[response]))
            # col nums of selected responses
            predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
            model_ols = LinearRegression()
            model_ols.fit(X[:, predcols], y)
            y_pred = model_ols.predict(X[:, predcols])
            print ("In-sample OLS R-squared: %.2f%%" % (100 * r2_score(y, y_pred)))
            print("---")
            
    return coef_dict

coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'), verbose=True)


LASSO variables selected for Food.lead: 
['10year', 'Mines.12m']
Running OLS for Food.lead against ['10year', 'Mines.12m']
In-sample OLS R-squared: 2.26%
---
LASSO variables selected for Beer.lead: 
['Food', 'Clths', 'Coal', '3month', '10year', 'month_dummy_02', 'month_dummy_04', 'month_dummy_05', 'month_dummy_07', 'month_dummy_08', 'Beer.3m', 'Hlth.3m', 'Util.3m', 'Mines.12m', 'Coal.12m', 'Servs.12m', '3month.12m']
Running OLS for Beer.lead against ['Food', 'Clths', 'Coal', '3month', '10year', 'month_dummy_02', 'month_dummy_04', 'month_dummy_05', 'month_dummy_07', 'month_dummy_08', 'Beer.3m', 'Hlth.3m', 'Util.3m', 'Mines.12m', 'Coal.12m', 'Servs.12m', '3month.12m']
In-sample OLS R-squared: 8.47%
---
LASSO variables selected for Smoke.lead: 
['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', '3month', 'month_dummy_05', 'month_dummy_07', 'month_dummy_08', 'month_dummy_09', 'month_dummy_10', 'month_dummy_11', 'Food.3m', 'Beer.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Util.3m', '

LASSO variables selected for BusEq.lead: 
['Books', 'Util', '3month', 'month_dummy_05', 'month_dummy_08', 'month_dummy_10', 'month_dummy_12', '10year.3m', 'Smoke.12m', 'Txtls.12m', 'Steel.12m', 'Telcm.12m', '3month.12m']
Running OLS for BusEq.lead against ['Books', 'Util', '3month', 'month_dummy_05', 'month_dummy_08', 'month_dummy_10', 'month_dummy_12', '10year.3m', 'Smoke.12m', 'Txtls.12m', 'Steel.12m', 'Telcm.12m', '3month.12m']
In-sample OLS R-squared: 8.91%
---
LASSO variables selected for Paper.lead: 
['Books', 'Clths', 'ElcEq', 'Carry', 'Coal', 'Oil', 'BusEq', 'Rtail', 'Fin', '3month', '10year', 'month_dummy_03', 'month_dummy_04', 'month_dummy_05', 'month_dummy_08', 'month_dummy_10', 'month_dummy_11', 'Food.3m', 'Coal.3m', 'Util.3m', 'Paper.3m', '10year.3m', 'Games.12m', 'Clths.12m', 'Steel.12m', 'FabPr.12m', 'Carry.12m', 'Coal.12m', '10year.12m', 'curve.12m']
Running OLS for Paper.lead against ['Books', 'Clths', 'ElcEq', 'Carry', 'Coal', 'Oil', 'BusEq', 'Rtail', 'Fin', '3month',

In [7]:
def predict_with_subsets(X, Y, create_model, coef_dict, verbose=False):
    """similar output to subset_selection, but pass a model function and a dictionary, compute avg R-squared"""
    global responses
    global response_reverse_dict
    
    model = create_model()
    
    scores = []
    for response in responses:
        y = Y[:,response_reverse_dict[response]]

#        print("LASSO variables selected for %s: " % pred)
#        print(coef_dict[pred])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response)
 #           print("---")
            continue
        # fit model vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        #print("Running model for " + pred + " against " + str(coef_dict[pred]))
        # col nums of selected predictors
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        model.fit(X[:, predcols], y)
        y_pred = model.predict(X[:, predcols])
        score = r2_score(y, y_pred)
        scores.append(score)
        if verbose:
            print ("In-sample R-squared: %.4f%% for %s against %s" % (score*100, response, str(coef_dict[response])))
#        print("---")
    
    if verbose:
        print("Mean R-squared: %.4f%%" % (100 * np.mean(np.array(scores))))
    return np.mean(np.array(scores))
    

predict_with_subsets(X, Y, LinearRegression, coef_dict, verbose=True)


In-sample R-squared: 2.2604% for Food.lead against ['10year', 'Mines.12m']
In-sample R-squared: 8.4721% for Beer.lead against ['Food', 'Clths', 'Coal', '3month', '10year', 'month_dummy_02', 'month_dummy_04', 'month_dummy_05', 'month_dummy_07', 'month_dummy_08', 'Beer.3m', 'Hlth.3m', 'Util.3m', 'Mines.12m', 'Coal.12m', 'Servs.12m', '3month.12m']
In-sample R-squared: 15.4921% for Smoke.lead against ['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', '3month', 'month_dummy_05', 'month_dummy_07', 'month_dummy_08', 'month_dummy_09', 'month_dummy_10', 'month_dummy_11', 'Food.3m', 'Beer.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Util.3m', 'Servs.3m', 'Paper.3m', 'Other.3m', 'Food.12m', 'Smoke.12m', 'Games.12m', 'Hshld.12m', 'Hlth.12m', 'FabPr.12m', 'Paper.12m', '10year.12m']
In-sample R-squared: 8.6195% for Games.lead against ['Books', 'Clths', 'Coal', 'Fin', '3month', 'curve', 'month_dummy_05', 'month_dummy_06', 'month_dummy_08', 'month_dummy_12', 'Oil.12m', '3month.12m']
In-sample R-

0.09028732911791308

In [8]:
coef_dict_all = {}
for response in responses:
    coef_dict_all[response] = predictors
predict_with_subsets(X, Y, LinearRegression, coef_dict_all, verbose=True)


In-sample R-squared: 18.7937% for Food.lead against ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other', '3month', '10year', 'curve', 'month_dummy_00', 'month_dummy_01', 'month_dummy_02', 'month_dummy_03', 'month_dummy_04', 'month_dummy_05', 'month_dummy_06', 'month_dummy_07', 'month_dummy_08', 'month_dummy_09', 'month_dummy_10', 'month_dummy_11', 'month_dummy_12', 'Mkt-RF', 'Food.3m', 'Beer.3m', 'Smoke.3m', 'Games.3m', 'Books.3m', 'Hshld.3m', 'Clths.3m', 'Hlth.3m', 'Chems.3m', 'Txtls.3m', 'Cnstr.3m', 'Steel.3m', 'FabPr.3m', 'ElcEq.3m', 'Autos.3m', 'Carry.3m', 'Mines.3m', 'Coal.3m', 'Oil.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'BusEq.3m', 'Paper.3m', 'Trans.3m', 'Whlsl.3m', 'Rtail.3m', 'Meals.3m', 'Fin.3m', 'Other.3m', '3month.3m', '10year.3m', 'curve.3m', 'Mkt-RF.3m', 'Food.12m',

0.21552597958630587

In [9]:
def fit_predict(X, Y, model, coef_dict="all", npredict=1):
    """for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final row
    if npredict=12, fit using n-12 rows, return prediction using X for final 12 rows
    coef_dict = None -> perform subset selection at each timestep using subset_selection()
    coef_dict = 'all' -> use all predictors"""
    
    global responses
    global response_reverse_dict
    
    # keep last row to predict against
    X_predict = X[-npredict:]
    nrows, ncols = X.shape
    X_predict = X_predict.reshape(npredict,ncols)
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]

    # if no coef_dict select predictors into coef_dict
    if coef_dict is None:
        coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))
    # if coef_dict == "all" use all predictors for each response        
    elif coef_dict == 'all':
        coef_dict = {}
        for response in responses:
            coef_dict[response]=predictors

    predictions = []
    for response in responses:
        if not coef_dict[response]:
            predictions.append([np.nan]*npredict)
            continue
        # column indexes to fit against each other
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        responsecol = response_reverse_dict[response]
        #print("fitting on" + str(X_fit[:, predcols].shape))
        model.fit(X_fit[:, predcols], Y_fit[:,responsecol])
        #print("predicting on" + str(X_predict[:, predcols].shape))
        y_pred = model.predict(X_predict[:,predcols])        
        predictions.append(y_pred)
        
    return np.array(predictions).transpose()

X = data.values[:,:npredictors]
Y = data.values[:, -nresponses:]
print(X[684])
print(Y[684])
predictions = fit_predict(X[:685], Y[:685], LinearRegression(), coef_dict, 1)
print(predictions)

[ 4.43        3.          5.39       -3.36        1.98        1.43
 -0.44        0.82        0.32       -1.27        0.16       -2.14
 -0.15       -0.4         4.16        1.4        -1.8        -9.68
  2.17        3.61        4.65       -0.19        2.07        1.86
  0.84        2.34       -0.98        0.58        3.8         2.57
  0.06        0.35        1.98        0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.          1.82       -0.10333333
 -1.47        1.62        2.03333333 -0.05       -1.90333333 -1.46666667
 -1.75333333  1.97333333 -1.28666667  2.          5.24333333  2.73
  1.06        2.58666667  4.13333333  3.52333333  0.80333333  2.92333333
  0.04333333  2.68333333 -0.25        0.74        0.19333333  4.31666667
  1.15       -0.52666667  2.36333333  6.01333333  3.44333333  0.07333333
  0.28666667  1.7         1.55333333  0.59583333  0.49083333  1.31416667
  1.43416667  0.43        0.49

In [10]:
# first iteration will train up to including 196911
# will use 196912 to predict 197001
# 19701 will be first month of performance to use
# train on first 121 months up to 196912 (0:120), put first prediction in P[121] (122nd row)
# first month of performance will be 197002
FIRST_TRAIN_MONTHS = 121
FIRST_PREDICT_MONTH = FIRST_TRAIN_MONTHS # This is stupid but keeps my head straight

print(X[FIRST_TRAIN_MONTHS])
print(data.iloc[FIRST_TRAIN_MONTHS][:30])

[-3.34000000e+00 -1.95000000e+00 -7.59000000e+00 -7.76000000e+00
 -1.20500000e+01 -7.50000000e+00 -5.69000000e+00 -7.71000000e+00
 -7.37000000e+00 -5.26000000e+00 -9.84000000e+00 -6.31000000e+00
 -7.15000000e+00 -6.89000000e+00 -9.35000000e+00 -1.24900000e+01
 -2.34000000e+00 -7.70000000e-01 -1.21600000e+01 -4.83000000e+00
 -3.16000000e+00 -1.11700000e+01 -9.73000000e+00 -8.89000000e+00
 -8.17000000e+00 -8.28000000e+00 -6.31000000e+00 -1.31200000e+01
 -9.78000000e+00 -6.20000000e+00  5.00000000e-02  1.40000000e-01
 -8.00000000e-02  0.00000000e+00  1.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -8.10000000e+00 -2.28666667e+00
 -2.18000000e+00 -3.36000000e+00 -7.00333333e+00 -6.82000000e+00
 -3.37666667e+00 -5.32666667e+00 -1.41000000e+00 -5.58666667e+00
 -5.43333333e+00 -6.02666667e+00 -4.45000000e+00 -4.68333333e+00
 -4.67666667e+00 -5.93666

In [11]:
# make predictions for all months in backtest period
# initial train_months = 120 -> train first model on 120 rows
# first prediction will be in P[120] (121st row)
# step = 6 -> predict following 6 rows, then step forward 6 months at a time
# initialize global predictions matrix P

def run_backtest(X, Y, create_model, coef_dict=None, initial_train_months=0, 
                 minmaxscale=False, standardscale=False, step=1, verbose=False):
    global P
    P = np.zeros_like(Y)

    count = 0
    nrows, ncols = Y.shape
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")
        
    # create list of steps
    month_indexes = list(range(initial_train_months+step, nrows, step))
    # force an iteration for last (nrows % step) rows
    if month_indexes[-1] < nrows: 
        month_indexes.append(nrows)
    if verbose:
        print("Initial train rows = %d" % initial_train_months)
        print ("Steps: " + str(month_indexes))

    model = create_model()
    
    for month_index in month_indexes:
        # if month_index =121 and step =1
        # train on 0:119 (120 rows)
        # fit on 120 (121st row)
        # if month_index = 1000 and step = 10
        # fit on 0:989, predict 990:999
        if verbose:
            print("Training on first %d rows (%d:%d), putting prediction in rows %s" % (month_index-step, 0, month_index-step-1, str(range(month_index-step,month_index))))
        predictions = fit_predict(Xscale[:month_index], 
                                  Yscale[:month_index], 
                                  model,
                                  coef_dict,
                                  step)
        #990
        first_pred_row = month_index-step
        for row_index in range(step):
            P[first_pred_row+row_index] = predictions[row_index]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, (X.shape[0]-initial_train_months+1)))
            sys.stdout.flush()
    print("")

    msetemp = (P[initial_train_months:]-Yscale[initial_train_months:])**2
    #remove nans
    msetemp = msetemp[~np.isnan(msetemp)]
    mse = np.mean(msetemp)
    print("MSE across all predictions: %.4f" % mse)
    print("Variance: %.4f" % (np.mean(Yscale[initial_train_months:]**2)))
    print("R-squared: %.4f" % (1- mse/(np.mean(Yscale[initial_train_months:]**2))))
     


In [12]:
def gen_returns(Y, P, first_pred_month, start_date='01/01/1970', freq='M', verbose=False):
    #TODO: more general version
    #take an indicator (P), a function to generate portfolio based on indicator, universe returns (Y), return portfolio returns

    global R
    R = np.zeros(P.shape[0])
    nrows, ncols = P.shape
    numstocks = 6 # top quintile (and bottom)

    indcount = [0 for response in responses]
    longcount = [0 for response in responses]
    shortcount = [0 for response in responses]
        
    for month_index in range(first_pred_month, nrows):
        # get indexes of sorted smallest to largest
        # leftmost 6
        # ignore nan
        short_sort_array = [999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(short_sort_array)
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_sort_array = [-999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(long_sort_array)
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        return_month = month_index + 1
        if verbose:
            print("Longs for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in long_indexes])))
            print("Shorts for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in short_indexes])))
            
        if return_month < nrows: # last row has a prediction for following month but no following month
            R[return_month] = np.mean(X[return_month, long_indexes])/2 - np.mean(X[return_month, short_indexes])/2
            # count occurrences of each industry
            for i in short_indexes:
                indcount[i]+=1
                shortcount[i]+=1
            for i in long_indexes:
                indcount[i]+=1
                longcount[i]+=1

    for response in responses:
        i = response_reverse_dict[response]
        print("%s: long %d times, short %d times, total %d times" % (response, longcount[i], shortcount[i], indcount[i]))
        
    results = R[first_pred_month:]

    index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [13]:
# run linear regression for baseline
run_backtest(X, Y, LinearRegression, coef_dict, initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)

# this is even better than the paper (Sharpe 0.97)
# clearly there is impact from snooping, but we knew that
# (paper presumably uses theoretically snoop-free LASSO threshold)
# add a bunch of variables, select good ones using whole period, use the good ones in backtest
# more variables you add, better the result, more snooping effect, LASSO info leaks back into earlier performance

# OOS MSE of 39.3998 is worse, suggests overfitting
# But performance is better, suggests no overfitting. 
# Relationship between MSE/R-squared and performance seems tenuous, which is a little surprising.


................................................................................
10:23:23 Still training 80 of 577
................................................................................
10:23:24 Still training 160 of 577
................................................................................
10:23:26 Still training 240 of 577
................................................................................
10:23:27 Still training 320 of 577
................................................................................
10:23:29 Still training 400 of 577
................................................................................
10:23:30 Still training 480 of 577
................................................................................
10:23:32 Still training 560 of 577
................
MSE across all predictions: 39.3998
Variance: 39.5535
R-squared: 0.0039
Food.lead: long 81 times, short 77 times, total 158 times
Beer.lead: long 118 times, short 87 times,

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0969051
yearly_vol,0.105352
yearly_sharpe,0.985345
max_drawdown,-0.0712009
sortino,0.978702


In [14]:
# double check results_post_LASSO
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results_post_LASSO = R[FIRST_PREDICT_MONTH:]
print(len(results_post_LASSO))
#print(results_post_LASSO)
print(np.mean(results_post_LASSO))
print(np.std(results_post_LASSO) * np.sqrt(12))
print(np.prod(1 + results_post_LASSO / 100))
print(np.prod(1 + results_post_LASSO / 100) ** (12.0/len(results_post_LASSO)))

576
0.7853197337962963
5.616408382268413
84.08229346498587
1.0967256769204723


In [15]:
# run performance chart
perf_post_LASSO = 100 * np.cumprod(1 + results_post_LASSO / 100)

def mychart(args, names=None):
    x_coords = np.linspace(1970, 2016, args[0].shape[0])
    
    plotdata = []
    for i in range(len(args)):
        tracelabel = "Trace %d" % i
        if names:
                tracelabel=names[i]
        plotdata.append(Scatter(x=x_coords,
                                y=args[i].reshape(-1),
                                mode = 'line',
                                name=tracelabel))    

    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    
mychart([perf_post_LASSO],["Post-LASSO"])

In [16]:
# pass coef_dict as None
# fit_predict will do subset selection at each timestep using data it trains on
run_backtest(X, Y, LinearRegression, coef_dict=None, initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH)

# Returns not as good as with fewer variables (unsurprisingly)

................................................................................
10:25:00 Still training 80 of 577
................................................................................
10:26:06 Still training 160 of 577
................................................................................
10:27:08 Still training 240 of 577
................................................................................
10:28:03 Still training 320 of 577
................................................................................
10:28:56 Still training 400 of 577
................................................................................
10:29:50 Still training 480 of 577
................................................................................
10:30:43 Still training 560 of 577
................
MSE across all predictions: 46.9536
Variance: 39.5535
R-squared: -0.1871
Food.lead: long 81 times, short 74 times, total 155 times
Beer.lead: long 99 times, short 111 times

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0379483
yearly_vol,0.0549741
yearly_sharpe,0.698329
max_drawdown,-0.152728
sortino,0.370122


In [17]:
results_LASSO_each_timestep = R[FIRST_PREDICT_MONTH:]
perf_LASSO_each_timestep = 100 * np.cumprod(1 + results_LASSO_each_timestep / 100)
mychart([perf_LASSO_each_timestep])

In [18]:
# pass coef_dict as 'all'
# fit_predict will use all predictors (no subset selection)
run_backtest(X, Y, LinearRegression, coef_dict='all', initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH)

................................................................................
10:31:01 Still training 80 of 577
................................................................................
10:31:06 Still training 160 of 577
................................................................................
10:31:12 Still training 240 of 577
................................................................................
10:31:18 Still training 320 of 577
................................................................................
10:31:24 Still training 400 of 577
................................................................................
10:31:31 Still training 480 of 577
................................................................................
10:31:38 Still training 560 of 577
................
MSE across all predictions: 85.8968
Variance: 39.5535
R-squared: -1.1717
Food.lead: long 96 times, short 73 times, total 169 times
Beer.lead: long 119 times, short 121 time

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0324746
yearly_vol,0.0714097
yearly_sharpe,0.504359
max_drawdown,-0.20242
sortino,0.285197


In [19]:
results_OLS = R[FIRST_PREDICT_MONTH:]
perf_OLS = 100 * np.cumprod(1 + results_OLS / 100)
mychart([perf_OLS])

In [20]:
mychart([perf_post_LASSO, perf_LASSO_each_timestep, perf_OLS],["Post-LASSO", "LASSO each timestep", "OLS"])

In [21]:
def walkforward_xval (X, Y, create_model, coef_dict_param="all", n_splits=5):

    start = time.time()

    # generate k-folds
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    model = create_model()
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("%s Training indexes 0 to %d" % (time.strftime("%H:%M:%S"), last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("%s Cross-validating indexes %d to %d" % (time.strftime("%H:%M:%S"), last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        if coef_dict_param is None:
            print("%s Performing LASSO subset selection on training set" % (time.strftime("%H:%M:%S")))
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'), verbose=False)
        elif coef_dict_param == 'all':
            # if coef_dict == "all" use all predictors for each response
            coef_dict = {}
            for response in responses:
                coef_dict[response]=predictors
        else:
            coef_dict = coef_dict_param
                
        mse_list = []
        
        for response in responses:
            predcols = [predictor_reverse_dict[indstr] for indstr in coef_dict[response]]
            if len(predcols) == 0:
                continue
            responsecol = response_reverse_dict[response]
            
            fit = model.fit(X_fit[:,predcols], Y_fit[:,responsecol])
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = fit.predict(X_xval[:,predcols])
            mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred))
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()             
        # mean mse over industry ys for this fold
        xval_score = np.mean(np.array(mse_list))            

        # choose model with lowest xval loss
        print ("\n%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    print ("Last Xval loss %f" % (xval_score))
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [22]:
# get baseline using linear regression
walkforward_xval (X, Y, LinearRegression, coef_dict_param=None)


10:31:39 Generate splits [140, 280, 419, 558, 697]
10:31:39 Starting training
10:31:39 Training indexes 0 to 279
10:31:39 Cross-validating indexes 280 to 418
10:31:39 Performing LASSO subset selection on training set
..............................
10:31:40 Xval MSE 31.112111
10:31:40 Training indexes 0 to 418
10:31:40 Cross-validating indexes 419 to 557
10:31:40 Performing LASSO subset selection on training set
..............................
10:31:41 Xval MSE 71.871868
10:31:41 Training indexes 0 to 557
10:31:41 Cross-validating indexes 558 to 696
10:31:41 Performing LASSO subset selection on training set
..............................
10:31:42 Xval MSE 69.084543
Last Xval loss 69.084543
Avg Xval loss 57.356174
--------------------------------------------------------------------------------


(57.35617401971416,
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))

In [23]:
# closure to wrap model creation function - return a function to create model
def create_model(n_hidden_layers, layer_size, reg_penalty):
    def create_func():
        return MLPRegressor(hidden_layer_sizes=tuple([layer_size]*n_hidden_layers),
                            alpha=reg_penalty,
                            activation='tanh',
                            max_iter=10000, 
                            tol=1e-10,
                            solver='lbfgs')
    return create_func

In [24]:
walkforward_xval (X, Y, create_model(3,2,1.0), coef_dict_param="all")

10:31:42 Generate splits [140, 280, 419, 558, 697]
10:31:42 Starting training
10:31:42 Training indexes 0 to 279
10:31:42 Cross-validating indexes 280 to 418
..............................
10:33:14 Xval MSE 62.052036
10:33:14 Training indexes 0 to 418
10:33:14 Cross-validating indexes 419 to 557
..............................
10:35:02 Xval MSE 87.201329
10:35:02 Training indexes 0 to 557
10:35:02 Cross-validating indexes 558 to 696
..............................
10:37:02 Xval MSE 78.015956
Last Xval loss 78.015956
Avg Xval loss 75.756440
--------------------------------------------------------------------------------


(75.75644031135504,
 MLPRegressor(activation='tanh', alpha=1.0, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(2, 2, 2), learning_rate='constant',
        learning_rate_init=0.001, max_iter=10000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='lbfgs', tol=1e-10, validation_fraction=0.1,
        verbose=False, warm_start=False))

In [25]:
def run_experiment(X, Y, 
                   n_hidden_layers, 
                   layer_size, 
                   reg_penalty,
                   coef_dict="all", 
                   minmaxscale=False, 
                   standardscale=False):
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")

    return walkforward_xval(Xscale, Yscale, 
                            create_model(n_hidden_layers, layer_size, reg_penalty), 
                            coef_dict_param=coef_dict)


In [26]:
# try many combos

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4, 8]
reg_penalties = [0.0, 0.0001, 0.01, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))
experiments = {}
        
for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))    
    score, model = run_experiment (X, Y, n_hidden_layers, layer_size, reg_penalty)
    experiments[key] = score


10:37:02 Running 48 experiments
10:37:02 Running experiment 1 of 48
10:37:02 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.000000
10:37:02 Generate splits [140, 280, 419, 558, 697]
10:37:02 Starting training
10:37:02 Training indexes 0 to 279
10:37:02 Cross-validating indexes 280 to 418
..............................
10:37:05 Xval MSE 33.701648
10:37:05 Training indexes 0 to 418
10:37:05 Cross-validating indexes 419 to 557
..............................
10:37:09 Xval MSE 69.823709
10:37:09 Training indexes 0 to 557
10:37:09 Cross-validating indexes 558 to 696
..............................
10:37:13 Xval MSE 65.725422
Last Xval loss 65.725422
Avg Xval loss 56.416927
--------------------------------------------------------------------------------
10:37:13 Running experiment 2 of 48
10:37:13 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.000100
10:37:13 Generate splits [140, 280, 419, 558, 697]
10:37:13 Starting training
10:37:13 Training indexes 0 to 279
10:37:

..............................
10:50:32 Xval MSE 99.454654
10:50:32 Training indexes 0 to 418
10:50:32 Cross-validating indexes 419 to 557
..............................
10:51:48 Xval MSE 140.001311
10:51:48 Training indexes 0 to 557
10:51:48 Cross-validating indexes 558 to 696
..............................
10:53:10 Xval MSE 115.625320
Last Xval loss 115.625320
Avg Xval loss 118.360429
--------------------------------------------------------------------------------
10:53:10 Running experiment 13 of 48
10:53:10 n_hidden_layers = 1, hidden_layer_size = 8, reg_penalty = 0.000000
10:53:10 Generate splits [140, 280, 419, 558, 697]
10:53:10 Starting training
10:53:10 Training indexes 0 to 279
10:53:10 Cross-validating indexes 280 to 418
..............................
10:54:12 Xval MSE 75.885579
10:54:12 Training indexes 0 to 418
10:54:12 Cross-validating indexes 419 to 557
..............................
10:55:41 Xval MSE 120.189973
10:55:41 Training indexes 0 to 557
10:55:41 Cross-validatin

..............................
11:17:34 Xval MSE 74.040990
11:17:34 Training indexes 0 to 557
11:17:34 Cross-validating indexes 558 to 696
..............................
11:18:16 Xval MSE 71.915938
Last Xval loss 71.915938
Avg Xval loss 61.693795
--------------------------------------------------------------------------------
11:18:16 Running experiment 24 of 48
11:18:16 n_hidden_layers = 2, hidden_layer_size = 2, reg_penalty = 1.000000
11:18:16 Generate splits [140, 280, 419, 558, 697]
11:18:16 Starting training
11:18:16 Training indexes 0 to 279
11:18:16 Cross-validating indexes 280 to 418
..............................
11:19:32 Xval MSE 66.042023
11:19:32 Training indexes 0 to 418
11:19:32 Cross-validating indexes 419 to 557
..............................
11:21:08 Xval MSE 88.190571
11:21:08 Training indexes 0 to 557
11:21:08 Cross-validating indexes 558 to 696
..............................
11:22:49 Xval MSE 82.494959
Last Xval loss 82.494959
Avg Xval loss 78.909185
---------------

..............................
12:11:18 Xval MSE 65.216032
Last Xval loss 65.216032
Avg Xval loss 55.117100
--------------------------------------------------------------------------------
12:11:18 Running experiment 35 of 48
12:11:18 n_hidden_layers = 3, hidden_layer_size = 1, reg_penalty = 0.010000
12:11:18 Generate splits [140, 280, 419, 558, 697]
12:11:18 Starting training
12:11:18 Training indexes 0 to 279
12:11:18 Cross-validating indexes 280 to 418
..............................
12:11:22 Xval MSE 31.505385
12:11:22 Training indexes 0 to 418
12:11:22 Cross-validating indexes 419 to 557
..............................
12:11:29 Xval MSE 69.363874
12:11:29 Training indexes 0 to 557
12:11:29 Cross-validating indexes 558 to 696
..............................
12:11:36 Xval MSE 65.301951
Last Xval loss 65.301951
Avg Xval loss 55.390403
--------------------------------------------------------------------------------
12:11:36 Running experiment 36 of 48
12:11:36 n_hidden_layers = 3, hidden

..............................
13:02:47 Xval MSE 76.722712
13:02:47 Training indexes 0 to 418
13:02:47 Cross-validating indexes 419 to 557
..............................
13:05:48 Xval MSE 119.316656
13:05:48 Training indexes 0 to 557
13:05:48 Cross-validating indexes 558 to 696
..............................
13:09:23 Xval MSE 114.475572
Last Xval loss 114.475572
Avg Xval loss 103.504980
--------------------------------------------------------------------------------
13:09:23 Running experiment 47 of 48
13:09:23 n_hidden_layers = 3, hidden_layer_size = 8, reg_penalty = 0.010000
13:09:23 Generate splits [140, 280, 419, 558, 697]
13:09:23 Starting training
13:09:23 Training indexes 0 to 279
13:09:23 Cross-validating indexes 280 to 418
..............................
13:12:05 Xval MSE 68.217569
13:12:05 Training indexes 0 to 418
13:12:05 Cross-validating indexes 419 to 557
..............................
13:15:06 Xval MSE 119.492364
13:15:06 Training indexes 0 to 557
13:15:06 Cross-validatin

In [27]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "loss"])
lossframe.sort_values(['loss'])

,n_hidden_layers,layer_size,reg_penalty,loss
30,3,1,0.0000,55.082895
9,3,1,0.0001,55.117100
33,3,1,0.0100,55.390403
42,2,1,0.0001,55.582625
36,2,1,0.0000,55.739897
43,2,1,0.0100,56.078172
37,1,1,0.0001,56.195519
35,1,1,0.0000,56.416927
26,1,1,0.0100,56.687750
44,2,2,0.0001,60.183724


In [28]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


,loss
n_hidden_layers,
1,79.662678
2,75.874612
3,76.006156


In [29]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())


,loss
layer_size,
1,56.969173
2,65.984975
4,81.187155
8,104.583292


In [30]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())


,loss
reg_penalty,
0.0000,72.414035
0.0001,73.100039
0.0100,74.130586
1.0000,89.079934


In [31]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[y_labels], columns=[x_labels], values=['loss'])
#    print(pivot)
    # specify labels as strings, to force plotly to use a discrete axis
#    print(pivot.columns.levels[1]).values
#    print(lossframe[x_labels].dtype)
    
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
#    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

In [32]:
plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" layers", y_suffix=" units")

In [33]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix=" layers", y_suffix=" rp")


In [34]:
plot_matrix(lossframe, "layer_size", "reg_penalty", x_suffix=" units", y_suffix=" rp")


In [35]:
# these results are not very good, ~same as LinearRegression, but try best one
# 1-unit layers is not much of a NN but allows some non-linearity, let's see how it performs

run_backtest(X, Y, create_model(3, 1, 0.01), 
             coef_dict='all', 
             initial_train_months=FIRST_TRAIN_MONTHS)

#  better OOS MSE  (~41) than linear regression (~45) but much worse Sharpe

................................................................................
13:33:03 Still training 80 of 577
................................................................................
13:38:31 Still training 160 of 577
................................................................................
13:44:15 Still training 240 of 577
................................................................................
13:51:04 Still training 320 of 577
................................................................................
13:58:22 Still training 400 of 577
................................................................................
14:05:35 Still training 480 of 577
................................................................................
14:12:45 Still training 560 of 577
................
MSE across all predictions: 41.5920
Variance: 39.5535
R-squared: -0.0515


In [36]:
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)


Food.lead: long 96 times, short 67 times, total 163 times
Beer.lead: long 111 times, short 90 times, total 201 times
Smoke.lead: long 190 times, short 86 times, total 276 times
Games.lead: long 197 times, short 164 times, total 361 times
Books.lead: long 131 times, short 120 times, total 251 times
Hshld.lead: long 78 times, short 101 times, total 179 times
Clths.lead: long 147 times, short 141 times, total 288 times
Hlth.lead: long 110 times, short 90 times, total 200 times
Chems.lead: long 56 times, short 135 times, total 191 times
Txtls.lead: long 135 times, short 121 times, total 256 times
Cnstr.lead: long 77 times, short 108 times, total 185 times
Steel.lead: long 64 times, short 194 times, total 258 times
FabPr.lead: long 57 times, short 98 times, total 155 times
ElcEq.lead: long 116 times, short 96 times, total 212 times
Autos.lead: long 81 times, short 147 times, total 228 times
Carry.lead: long 160 times, short 117 times, total 277 times
Mines.lead: long 162 times, short 115 ti

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0143823
yearly_vol,0.0388984
yearly_sharpe,0.391569
max_drawdown,-0.161779
sortino,0.160059


In [37]:
# use keras instead of sklearn MLPRegressor
# wrap keras model in a class 
# multioutput 30 predictions simultaneously to speed up 
# fit takes a list of response ys, predict returns a list of y_predict arrays
# no coef_dict
INPUT_DIM = X.shape[1]
print(INPUT_DIM)
OUTPUT_DIM = len(responses) # 30
BATCH_SIZE = 32
EPOCHS=500

class KerasBacktestModel(object):

    def __init__(self, 
                 n_hidden_layers = 2,
                 hidden_layer_size = 32,
                 reg_penalty = 0.0001,
                 verbose=True):
        main_input = Input(shape=(INPUT_DIM,),
                           dtype='float32', 
                           name='main_input')
        lastlayer=main_input
        
        for i in range(n_hidden_layers):
            if verbose:
                print("layer %d size %d, reg_penalty %.8f" % (i + 1, 
                                                              hidden_layer_size, 
                                                              reg_penalty, 
                                                             ))
            lastlayer = Dense(units = hidden_layer_size, 
                              activation = 'relu',
                              kernel_initializer = keras.initializers.glorot_uniform(),
                              kernel_regularizer=keras.regularizers.l1(reg_penalty),
                              name = "Dense%02d" % i)(lastlayer)
            
        outputs = []
        for i in range(OUTPUT_DIM):
            # OUTPUT_DIM outputs
            outputs.append(Dense(1,
                                 activation='linear',
                                 name='output%02d' % i)(lastlayer)
                          )
            
        self.model = Model(inputs=[main_input], outputs=outputs)
        if verbose:
            print(self.model.summary())
            
        self.model.compile(loss="mse", optimizer="rmsprop", loss_weights=[1.]*OUTPUT_DIM)
        
    def fit(self, X, Y, epochs=EPOCHS):
        # convert Y to list of ys
        Y_list = [Y[:,i] for i in range(OUTPUT_DIM)]
        return self.model.fit(X,
                              Y_list,
                              batch_size=BATCH_SIZE,
                              epochs=epochs,
                              verbose=False)
    
    def predict(self, X):
        y_list = self.model.predict(X)
        # convert list of ys to Y array
        npreds=len(y_list[0])
        Y_pred = [y.reshape(npreds) for y in y_list]
        Y_pred = np.array(Y_pred).transpose()    
        return Y_pred
    
    def save(self, modelname):
        self.model.save("%s.h5" % modelname)
        self.model.save_weights("%s_weights.h5" % modelname)


115


In [38]:
def create_keras_model(n_hidden_layers, layer_size, reg_penalty, verbose=False):
    def create_func():
        return KerasBacktestModel(n_hidden_layers = n_hidden_layers,
                                  hidden_layer_size = layer_size,
                                  reg_penalty = reg_penalty,
                                  verbose=verbose)
    return create_func


In [39]:
def fit_predict_keras(X, Y, model, epochs=EPOCHS, npredict=1, verbose=False):
    """simpler fit_predict, no coef_dict, fits all at once, specifies epochs 
    for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final month
    if npredict=26, fit using n-26 rows, return prediction using X for final 26 months"""
    
    nrows = X.shape[0]
    if verbose:
        print("Fit on %d rows 0 to %d" % (nrows-npredict, nrows-npredict-1))
        print("Predict on %d rows %d to %d" % (npredict, nrows-npredict, nrows-1))
        
    # keep last rows to predict against
    X_predict = X[-npredict:]
    X_predict = X_predict.reshape(npredict,X.shape[1])
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]
    
    fit = model.fit(
        X_fit,
        Y_fit,
        epochs=epochs
    )
    
    return model.predict(X_predict)

print("%s Start fit" % (time.strftime("%H:%M:%S")))
keras_create_model=create_keras_model(1, 4, 0.01, verbose=True)
keras_model=keras_create_model()
predictions = fit_predict_keras(X, Y, keras_model, npredict=3, epochs=3)
print("%s End fit" % (time.strftime("%H:%M:%S")))

predictions

14:14:19 Start fit
layer 1 size 4, reg_penalty 0.01000000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 115)          0                                            
__________________________________________________________________________________________________
Dense00 (Dense)                 (None, 4)            464         main_input[0][0]                 
__________________________________________________________________________________________________
output00 (Dense)                (None, 1)            5           Dense00[0][0]                    
__________________________________________________________________________________________________
output01 (Dense)                (None, 1)            5           Dense00[0][0]                    
___________________________________________________

array([[ 0.00447078, -0.06241774,  0.0601501 ,  0.05172196, -0.01567799,
         0.04844666,  0.06786828,  0.00254934,  0.00330035,  0.04548017,
        -0.04128596,  0.04233812,  0.04413079, -0.0620181 , -0.01076399,
         0.03333197,  0.02794632,  0.03293885, -0.06063766,  0.0671836 ,
         0.02166007, -0.03212909,  0.03131388,  0.06512874,  0.05597452,
         0.05104017,  0.06908502,  0.06508877, -0.02960017, -0.04344311],
       [-1.0006152 ,  0.95296884, -0.39142352, -0.3085181 ,  1.1294794 ,
         0.10506582, -1.1294491 ,  0.14906001, -0.5099676 ,  0.13546406,
        -0.5364592 ,  0.57708305,  0.08593363,  0.62099826,  1.1378344 ,
        -0.31968063, -0.3739746 , -1.0447155 ,  1.1576296 , -0.6273625 ,
         1.0001388 , -0.8046274 ,  0.15319619,  0.09695688,  1.2281677 ,
         0.86195004, -0.8967417 ,  0.19920772,  1.1215547 ,  0.7797885 ],
       [ 0.52980787,  0.22713563,  0.10913628, -0.5600708 , -0.07780499,
         0.502563  ,  0.23636425, -0.03123635,  0

In [40]:
EPOCHS=500
def walkforward_xval_keras (X, Y, create_model, n_splits=5, epochs=EPOCHS):
    ### no coef_dict, fit all at once

    # generate k-folds
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    model = create_model()
    
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("%s Training indexes 0 to %d" % (time.strftime("%H:%M:%S"), last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        
        # xval from last_train_index to last_xval_index
        print("%s Cross-validating indexes %d to %d" % (time.strftime("%H:%M:%S"), last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        fit = model.fit(X_fit, Y_fit, epochs=epochs)
        # evaluate ... run prediction, calc MSE by industry, and average
        Y_xval_pred = model.predict(X_xval)
        
        mse_list = [mean_squared_error(Y_xval[:,i], Y_xval_pred[:,i]) for i in range(OUTPUT_DIM)]
        xval_score = np.mean(np.array(mse_list))            

        print ("%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("%s Avg Xval loss %f" % (time.strftime("%H:%M:%S"), avg_loss))
    return (avg_loss, model)


In [41]:

walkforward_xval_keras(X, Y, 
                       create_keras_model(n_hidden_layers=3,
                                          layer_size=2,
                                          reg_penalty=0.001),
                      epochs=5)

14:14:29 Generate splits [140, 280, 419, 558, 697]
14:14:29 Starting training
14:14:30 Training indexes 0 to 279
14:14:30 Cross-validating indexes 280 to 418
14:14:38 Xval MSE 33.074277
14:14:38 Training indexes 0 to 418
14:14:38 Cross-validating indexes 419 to 557
14:14:45 Xval MSE 40.660880
14:14:45 Training indexes 0 to 557
14:14:45 Cross-validating indexes 558 to 696
14:14:55 Xval MSE 42.046247
14:14:55 Avg Xval loss 38.593801


(38.59380124801013, <__main__.KerasBacktestModel at 0x7fb35d783210>)

In [42]:
# run an experiment with walk-forward cross-validation

EPOCHS = 50

def run_experiment(X, Y, 
                   n_hidden_layers, 
                   layer_size, 
                   reg_penalty,
                   minmaxscale=False, 
                   standardscale=False,
                   epochs=EPOCHS):
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")

    return walkforward_xval_keras(Xscale, Yscale,
                                  create_keras_model(n_hidden_layers=n_hidden_layers,
                                                     layer_size=layer_size,
                                                     reg_penalty=reg_penalty),
                                  epochs=epochs)


In [43]:
run_experiment(X, Y, 3, 2, .001, epochs=50)

14:14:55 Generate splits [140, 280, 419, 558, 697]
14:14:55 Starting training
14:14:56 Training indexes 0 to 279
14:14:56 Cross-validating indexes 280 to 418
14:15:48 Xval MSE 33.000294
14:15:48 Training indexes 0 to 418
14:15:48 Cross-validating indexes 419 to 557
14:17:02 Xval MSE 41.294772
14:17:02 Training indexes 0 to 557
14:17:02 Cross-validating indexes 558 to 696
14:18:39 Xval MSE 46.010263
14:18:39 Avg Xval loss 40.101776


(40.101776224312374, <__main__.KerasBacktestModel at 0x7fb32f41c5d0>)

In [44]:
# improvement in walkforward MSE vs. LinearRegression (40ish vs. 54)
# run a lot of experiments in big xval loop to pick best hyperparameters

MODELPREFIX = "FFNN"
EPOCHS=40

n_hiddens = [1, 2, 3]
layer_sizes = [2, 4, 8]
reg_penalties = [0.0, 0.001, 0.01, 0.1]

hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    score, model = run_experiment(X, Y,
                                  n_hidden_layers = n_hidden_layers,
                                  layer_size = layer_size,
                                  reg_penalty = reg_penalty,
                                  epochs=EPOCHS
                                 )
    experiments[key] = score 
    modelname = "%s_%.6f_%d_%d_%.6f" % (MODELPREFIX, score, n_hidden_layers, layer_size, reg_penalty)
    print("%s Saving %s.h5" % (time.strftime("%H:%M:%S"), modelname))
    print("--------------------------------------------------------------------------------")
    model.save(modelname)


14:18:39 Running 36 experiments
14:18:39 Running experiment 1 of 36
14:18:39 Generate splits [140, 280, 419, 558, 697]
14:18:39 Starting training
14:18:40 Training indexes 0 to 279
14:18:40 Cross-validating indexes 280 to 418
14:19:22 Xval MSE 32.725985
14:19:22 Training indexes 0 to 418
14:19:22 Cross-validating indexes 419 to 557
14:20:20 Xval MSE 41.823374
14:20:20 Training indexes 0 to 557
14:20:20 Cross-validating indexes 558 to 696
14:21:35 Xval MSE 42.594333
14:21:35 Avg Xval loss 39.047897
14:21:35 Saving FFNN_39.047897_1_2_0.000000.h5
--------------------------------------------------------------------------------
14:21:38 Running experiment 2 of 36
14:21:38 Generate splits [140, 280, 419, 558, 697]
14:21:38 Starting training
14:21:38 Training indexes 0 to 279
14:21:38 Cross-validating indexes 280 to 418
14:22:22 Xval MSE 32.973129
14:22:22 Training indexes 0 to 418
14:22:22 Cross-validating indexes 419 to 557
14:23:22 Xval MSE 43.376830
14:23:22 Training indexes 0 to 557
14:2

15:02:27 Xval MSE 41.968372
15:02:27 Avg Xval loss 41.272698
15:02:27 Saving FFNN_41.272698_2_2_0.001000.h5
--------------------------------------------------------------------------------
15:02:39 Running experiment 15 of 36
15:02:39 Generate splits [140, 280, 419, 558, 697]
15:02:39 Starting training
15:02:39 Training indexes 0 to 279
15:02:39 Cross-validating indexes 280 to 418
15:03:27 Xval MSE 33.598398
15:03:27 Training indexes 0 to 418
15:03:27 Cross-validating indexes 419 to 557
15:04:26 Xval MSE 42.046660
15:04:26 Training indexes 0 to 557
15:04:26 Cross-validating indexes 558 to 696
15:05:44 Xval MSE 43.270908
15:05:44 Avg Xval loss 39.638655
15:05:44 Saving FFNN_39.638655_2_2_0.010000.h5
--------------------------------------------------------------------------------
15:05:57 Running experiment 16 of 36
15:05:57 Generate splits [140, 280, 419, 558, 697]
15:05:57 Starting training
15:05:57 Training indexes 0 to 279
15:05:57 Cross-validating indexes 280 to 418
15:06:45 Xval MS

15:51:46 Xval MSE 42.153544
15:51:46 Training indexes 0 to 557
15:51:46 Cross-validating indexes 558 to 696
15:53:06 Xval MSE 45.367837
15:53:06 Avg Xval loss 40.220106
15:53:06 Saving FFNN_40.220106_3_2_0.100000.h5
--------------------------------------------------------------------------------
15:53:29 Running experiment 29 of 36
15:53:29 Generate splits [140, 280, 419, 558, 697]
15:53:29 Starting training
15:53:29 Training indexes 0 to 279
15:53:29 Cross-validating indexes 280 to 418
15:54:22 Xval MSE 33.304973
15:54:22 Training indexes 0 to 418
15:54:22 Cross-validating indexes 419 to 557
15:55:24 Xval MSE 43.261208
15:55:24 Training indexes 0 to 557
15:55:24 Cross-validating indexes 558 to 696
15:56:43 Xval MSE 51.437099
15:56:43 Avg Xval loss 42.667760
15:56:43 Saving FFNN_42.667760_3_4_0.000000.h5
--------------------------------------------------------------------------------
15:57:06 Running experiment 30 of 36
15:57:06 Generate splits [140, 280, 419, 558, 697]
15:57:06 Starti

In [45]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]

lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty",
                                            "loss"])
lossframe.sort_values(['loss'])
# better than LinearRegression or MLPRegressor

,n_hidden_layers,layer_size,reg_penalty,loss
29,1,2,0.001,38.793025
33,2,2,0.000,38.836985
20,3,2,0.001,38.874619
24,3,2,0.000,39.009705
23,1,2,0.000,39.047897
10,1,2,0.010,39.089086
27,3,4,0.010,39.144828
0,2,4,0.010,39.224814
30,2,2,0.100,39.457655
22,2,2,0.010,39.638655


In [46]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


,loss
n_hidden_layers,
1,41.268269
2,42.674050
3,43.140966


In [47]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())

,loss
layer_size,
2,39.516660
4,41.590396
8,45.976228


In [48]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())

,loss
reg_penalty,
0.000,42.779156
0.001,42.766941
0.010,41.375338
0.100,42.522944


In [49]:
plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" layers", y_suffix=" units")


In [50]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix=" layers", y_suffix="rp")

In [51]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix="rp", y_suffix=" units")


In [52]:
EPOCHS=500

nrows = X.shape[0]
START=121

def run_backtest_keras(X, Y, arg_dict, startindex=0, epochs=EPOCHS, step=1, minmaxscale=False, standardscale=False):
    """create keras model; add step, to iteratively train, predict 12 months, train up to next 12 months """
    global P
    
    print("%s Starting backtest" % (time.strftime("%H:%M:%S")))
    P = np.zeros((Y.shape[0],OUTPUT_DIM))
    
    count = 0
    nrows = X.shape[0]

    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")
     
    model = create_keras_model(n_hidden_layers=arg_dict["n_hidden_layers"],
                               layer_size=arg_dict["hidden_layer_size"],
                               reg_penalty=arg_dict["reg_penalty"],
                               verbose=False)()
        
    for train_index in range(startindex, nrows, step):
        if train_index + step >= nrows:
            train_index = nrows-step
            
        fp_index = train_index + step # eg 1000 + 26 = 1026

        # fit on e.g. 0:999, predict 1000-1025
        predictions = fit_predict_keras(Xscale[:fp_index, :], 
                                        Yscale[:fp_index], 
                                        model,
                                        epochs=epochs,
                                        npredict=step)
        # store in 1000:1025 - lining up with future Xs
        for i in range(step):
            P[train_index + i]= predictions[i]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, nrows-startindex))
            sys.stdout.flush()

    msetemp = (P[startindex:]-Yscale[startindex:])**2
    #remove nans
    msetemp = msetemp[~np.isnan(msetemp)]
    mse = np.mean(msetemp)
    
    print("MSE across all predictions: %.4f" % mse)


In [53]:
def gen_returns(Y, P, first_pred_month, start_date='01/01/1970', freq='M', verbose=False):
    #TODO: more general version
    #take an indicator (P), a function to generate portfolio based on indicator, universe returns (Y), return portfolio returns

    global R
    R = np.zeros(P.shape[0])
    nrows, ncols = P.shape
    numstocks = 6 # top quintile (and bottom)

    indcount = [0 for response in responses]
    longcount = [0 for response in responses]
    shortcount = [0 for response in responses]
        
    for month_index in range(first_pred_month, nrows):
        # get indexes of sorted smallest to largest
        # leftmost 6
        # ignore nan
        short_sort_array = [999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(short_sort_array)
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_sort_array = [-999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(long_sort_array)
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        return_month = month_index + 1
        if verbose:
            print("Longs for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in long_indexes])))
            print("Shorts for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in short_indexes])))
            
        if return_month < nrows: # last row has a prediction for following month but no following month
            R[return_month] = np.mean(X[return_month, long_indexes])/2 - np.mean(X[return_month, short_indexes])/2
            # count occurrences of each industry
            for i in short_indexes:
                indcount[i]+=1
                shortcount[i]+=1
            for i in long_indexes:
                indcount[i]+=1
                longcount[i]+=1

    for response in responses:
        i = response_reverse_dict[response]
        print("%s: long %d times, short %d times, total %d times" % (response, longcount[i], shortcount[i], indcount[i]))
        
    results = R[first_pred_month:]

    index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [57]:
START=121
EPOCHS=40
STEP=1

arg_dict = {"n_hidden_layers" : 2,
            "hidden_layer_size" : 2,
            "reg_penalty" : 0.0,
            'verbose' : False
           }
     
#model = build_model(**arg_dict)
run_backtest_keras(X, Y, arg_dict, startindex=START, step=STEP, epochs=EPOCHS)


08:32:00 Starting backtest
................................................................................
09:05:57 Still training 80 of 576
................................................................................
09:51:55 Still training 160 of 576
................................................................................
10:52:04 Still training 240 of 576
................................................................................
12:06:58 Still training 320 of 576
................................................................................
13:35:15 Still training 400 of 576
................................................................................
15:17:46 Still training 480 of 576
................................................................................
17:14:41 Still training 560 of 576
................MSE across all predictions: 53.6569


In [58]:
gen_returns(X, P, START, verbose=False)


Food.lead: long 350 times, short 57 times, total 407 times
Beer.lead: long 60 times, short 101 times, total 161 times
Smoke.lead: long 441 times, short 15 times, total 456 times
Games.lead: long 95 times, short 436 times, total 531 times
Books.lead: long 10 times, short 26 times, total 36 times
Hshld.lead: long 62 times, short 5 times, total 67 times
Clths.lead: long 77 times, short 123 times, total 200 times
Hlth.lead: long 189 times, short 71 times, total 260 times
Chems.lead: long 0 times, short 89 times, total 89 times
Txtls.lead: long 14 times, short 6 times, total 20 times
Cnstr.lead: long 44 times, short 458 times, total 502 times
Steel.lead: long 7 times, short 492 times, total 499 times
FabPr.lead: long 18 times, short 299 times, total 317 times
ElcEq.lead: long 59 times, short 138 times, total 197 times
Autos.lead: long 0 times, short 105 times, total 105 times
Carry.lead: long 25 times, short 53 times, total 78 times
Mines.lead: long 115 times, short 7 times, total 122 times

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,-0.00384815
yearly_vol,0.0713379
yearly_sharpe,-0.0120277
max_drawdown,-0.30537
sortino,-0.0115582


In [56]:
results_MLP = R[FIRST_PREDICT_MONTH:]
perf_MLP = 100 * np.cumprod(1 + results_MLP / 100)
mychart([perf_MLP])